# Basic Plots of Single Pol Radar Data

- Reading and plotting radar data with `pyart`
    - São Roque (SR) (S Band, Single Pol) - variables *corrected_reflectivity*, *corrected_velocity*
    - Cases:
        - 2016-12-25
        - 2017-01-31
        - 2017-03-14
        - 2017-11-15
        - 2017-11-16

## Loading necessary packages

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import glob
from datetime import datetime

import pyart

%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Defining necessary functions

### `read_radar_data`
Using radar filenames, read the data and adjust some variables

In [2]:
def read_radar_data(filename):
    file = pyart.aux_io.read_gamic(filename)
    file_date = pd.to_datetime(file.time['units'][14:])
    
    file.fields['corrected_reflectivity']['standard_name'] = 'corrected_reflectivity'
    file.fields['corrected_velocity']['standard_name'] = 'corrected_velocity'
    
    print('File ' + filename + ' read!')
    return file

### `plot_horizontal_panel`
Using radar and hailpads data, plot horizontal view of fields with hailpad position

**WARNING**: Colorbar legends were modified from original Py-ART structure

`pyart/grapy/common.py` - line 93
```python
def generate_colorbar_label(standard_name, units):
    """ Generate and return a label for a colorbar. """
    return standard_name.replace('_', ' ').title() + '\n(' + units + ')'
```

In [3]:
def plot_horizontal_panel(file, hailpads, display_limits, shapefile, radar, save_to, sweep=1):
    #-- Defining date, sweep and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    file_sweep = file.fixed_angle['data'][sweep]
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
    
    #-- Start to plot
    display = pyart.graph.RadarMapDisplay(file)
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[9,2.75])
    fig.set_facecolor('w')
    xlim, ylim = display_limits
    
    #--- Reflectivity
    display.plot_ppi_map('corrected_reflectivity', sweep=sweep, ax=ax[0], shapefile=shapefile,
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 10, vmax = 70, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    if len(hailpad) == 2:
        display.plot_point(lat=hailpad['lat'][0], lon=hailpad['lon'][0], ax=ax[0], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
        display.plot_point(lat=hailpad['lat'][1], lon=hailpad['lon'][1], ax=ax[0], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
    else:
        display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[0], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
    #--- Doppler Velocity
    display.plot_ppi_map('corrected_velocity', sweep=sweep, ax=ax[1], shapefile=shapefile,
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = -15, vmax = 15, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    if len(hailpad) == 2:
        display.plot_point(lat=hailpad['lat'][0], lon=hailpad['lon'][0], ax=ax[1], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
        display.plot_point(lat=hailpad['lat'][1], lon=hailpad['lon'][1], ax=ax[1], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
    else:
        display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[1], symbol = 'ks', #-- Hailpad
                           markersize=10, markerfacecolor='None')
    
    #--- Common aspects
    plt.suptitle(radar.upper() + ' ' + str(round(file_sweep, 1)) + ' deg ' + str(file_date) + ' UTC', 
                 weight='bold', stretch='condensed', size='xx-large', y=1)
    
    #-- Saving the figure
    plt.savefig(save_to + radar + '_ppi_h_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                dpi=300, transparent=True, bbox_inches='tight')
    plt.close()
    
    return 'Plotting horizontal view for date ' + str(file_date)

### `plot_vertical_panel`
Using radar and hailpads data, plot vertical view of fields with hailpad position

**WARNING**: Colorbar legends were modified from original Py-ART structure

`pyart/grapy/common.py` - line 93
```python
def generate_colorbar_label(standard_name, units):
    """ Generate and return a label for a colorbar. """
    return standard_name.replace('_', ' ').title() + '\n(' + units + ')'
```

In [4]:
def plot_vertical_panel(file, hailpads, x_limits, radar, save_to):
    #-- Defining date and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
    
    #-- Plotting only when hailfall occurred
    if np.any(file_date.strftime('%H-%M') == hailpad['time_sr']):
        if len(hailpad) == 2:
            if file_date.strftime('%H-%M') == hailpad['time_sr'][0]:
                hailpad = hailpad.iloc[0]
            else:
                hailpad = hailpad.iloc[1]
        
        #--- Selecting cross-section
        file_cs = pyart.util.cross_section_ppi(file, [hailpad['azim_sr']])
        
        #--- Start to plot
        display = pyart.graph.RadarDisplay(file_cs)
        fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[6.5,2])
        fig.set_facecolor('w')
   
        #---- Reflectivity
        display.plot('corrected_reflectivity', vmin=10, vmax=70, title='', ax=ax[0], axislabels_flag=False)
        pnt = ax[0].scatter(x=hailpad['pos_sr'], y=1, marker='s', c='None', edgecolors='black') #-- Hailpad
        ax[0].grid(linestyle=':', c='k')
        
        #---- Doppler Velocity
        display.plot('corrected_velocity', vmin=-15, vmax=15, title='', ax=ax[1], axislabels_flag=False)
        pnt = ax[1].scatter(x=hailpad['pos_sr'], y=1, marker='s', c='None', edgecolors='black') #-- Hailpad
        ax[1].grid(linestyle=':', c='k')
        
        #--- Common aspects
        plt.suptitle(radar.upper() + ' ' + str(file_date) + ' UTC - Azimuth = ' + str(hailpad['azim_sr']) + ' deg', 
                     weight='bold', stretch='condensed', size='x-large', y=1.05)
        plt.xlim(x_limits)
        plt.ylim((0,18))
        
        #--- Manual labelling (outer_label doesn't work!)
        ax[0].set(xlabel='Distance from Radar (km)', ylabel='Distance above\nRadar (km)')
        ax[1].set(xlabel='Distance from Radar (km)', ylabel='')
        
        #--- Saving the figure
        plt.savefig(save_to + radar + '_ppi_v_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                    dpi=300, transparent=True, bbox_inches='tight')
        plt.close()
    
    return 'Plotting vertical view for date ' + str(file_date)

## Defining filepaths and custom variables

In [5]:
sr_filenames = open("files_sr_level0.txt").read().split('\n')
hailpad_data = pd.read_csv("../Data/GENERAL/hailpads_registry.txt", index_col=0)
shapefile = "../Data/GENERAL/shapefiles/sao_paulo"
save_path = "figures/ppis/sr/"
radar_name = "sr"

#-- Limits in horizontal view panel
#--- [[xlim], [ylim]]
limits = [[-47.5,-46.5], [-23.3,-22.5]]

#-- Limits (x axis) in vertical panel
#--- (xlim)
xlimits = (50,110)

print(hailpad_data)
print('sr_filenames length is: ' + str(len(sr_filenames)))

           time_cth time_sr       lat       lon  pos_cth  azim_cth  pos_sr  \
2016-12-25    19-40   19-40 -22.83092 -47.08152      141       307    85.0   
2017-01-31    19-24   19-30 -22.90958 -47.06725      135       304    78.0   
2017-03-14    18-27   18-30 -22.69160 -47.13110      155       310   101.0   
2017-03-14    19-57   20-00 -23.02940 -47.20541      140       297    63.0   
2017-11-15    21-50   24-00 -23.02940 -47.20541      142       296     NaN   
2017-11-16    18-20   18-20 -22.81405 -47.05641      141       308    88.0   

            azim_sr  
2016-12-25      1.6  
2017-01-31      1.6  
2017-03-14    358.5  
2017-03-14    350.0  
2017-11-15      NaN  
2017-11-16      2.7  
sr_filenames length is: 226


## Processing all the files in *sr_filenames*

Doing in small parts because Jupyter can't process large `for`